In [1]:
import os

In [2]:
%pwd

'/Users/abrar/ML-DS-Projects/Practice/End-to-End-ML-Project-with-MLOps/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/abrar/ML-DS-Projects/Practice/End-to-End-ML-Project-with-MLOps'

In [8]:
import pandas as pd

df = pd.read_csv('./artifacts/data_ingestion/winequality-red.csv')

In [9]:
df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [23]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):   
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self)-> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE, 
            all_schema=schema,
        )

        return data_validation_config

In [33]:
from src.mlProject import logger


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config    
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = True
            df = pd.read_csv(self.config.unzip_data_dir)
            
            if df.empty:
                validation_status = False
                logger.error("DataFrame is empty")
                self._save_validation_status(validation_status)
                return validation_status

            df_columns = set(df.columns)
            schema_columns = set(self.config.all_schema.keys())
            
            if not df_columns.issubset(schema_columns):
                validation_status = False
                missing_cols = df_columns - schema_columns
                logger.error(f"Missing columns in schema: {missing_cols}")
                self._save_validation_status(validation_status)
                return validation_status
            
            for column, expected_type in self.config.all_schema.items():
                actual_dtype = df[column].dtype
                if str(actual_dtype) != str(expected_type):
                    validation_status = False
                    logger.error(f"Column: {column} | Expected dtype: {expected_type} | Found dtype: {actual_dtype}")
                    self._save_validation_status(validation_status)
                    return validation_status
                        
            
            self._save_validation_status(validation_status)
            return validation_status

        except Exception as e:
            logger.error(f"Validation failed: {str(e)}")
            self._save_validation_status(False)
            raise e

    def _save_validation_status(self, status: bool):
        with open(self.config.STATUS_FILE, 'w') as f:
            f.write(f"Validation Status: {status}")

In [39]:
try:
    config_mgr = ConfigurationManager()
    data_validation_config = config_mgr.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-01-02 19:21:37,576: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-02 19:21:37,604: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 19:21:37,625: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-02 19:21:37,627: INFO: common: created directory at: artifacts]
[2025-01-02 19:21:37,629: INFO: common: created directory at: artifacts/data_validation]
